In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import cv2 as cv
import numpy as np
from PIL import Image
from torchvision import models
from torchvision.transforms import Normalize, ToTensor, Resize, Compose
from random import shuffle
import pandas as pd
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:

resmodel = models.resnet50(pretrained=True).to(device)
device
layers=list(resmodel.children())

In [ ]:
# layers


In [ ]:
category=["fastfading","gblur","jp2k","jpeg","wn"]

In [ ]:
l1=[]
import csv
with open("../input/live-wild/image_labeled_by_score.csv",newline='') as csvfile:
    read=csv.reader(csvfile,delimiter=" ")
    for x in read:
        l1+=x
        #print(str(x))
        
#print(l1)
l1.remove(l1[0])
#print(l1)
img_name=[]
mos=[]
for x in l1:
    y=x.split(",")
    #print(y)
    img_name+=[y[0]]
    mos+=[float(y[1])]
#print(img_name)
#print(mos)
dmos = dict(zip(img_name,mos))
#print(ds)
#print(dmos)

In [ ]:
# dmos = {}
# for filecategory in category:
#     filename = open(f'../input/mydata/Scores/{filecategory}.txt','r')

#     lines = filename.readlines()
# #     print(lines)

#     for line in lines:
#         input_line = line.split()
# #         print(obj)
#         if len(input_line)!=3:
        
        
#             continue
#         dmos[str(filecategory+'_'+input_line[1])] = float(input_line[2])/26.1
# #         print(dmos)
# # print(dmos)

In [ ]:
tfms = Compose([ToTensor(),
                Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

resmodel = models.resnet50(pretrained=True).to(device)
layers = list(resmodel.children())
model = nn.Sequential(*layers[:-1]).to(device).eval()

dataset = []

direc = '../input/live-wild/images'
for img in os.listdir(direc):
    if img == 'info.txt': continue
    try:
        image = Image.open(f'{direc}/{img}')
        image = tfms(image)
        
        with torch.no_grad():
            
            image = model(image.unsqueeze(0).to(device)).reshape(2048)
#             image = model(image.unsqueeze(0).to(device))
        x=dmos[img]
    
    
        dataset.append([image,x])
        
        #print(type(x))
    except:
        print(img)
        pass


In [ ]:
#from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
print(dataset)

In [ ]:

train_ds,test_ds=train_test_split(dataset,test_size=0.2)
#print(train_ds)

trainX = np.array([train_ds[i][0].cpu().numpy() for i in range(len(train_ds))])
trainY = np.array([train_ds[i][1] for i in range(len(train_ds))])

testX = np.array([test_ds[i][0].cpu().numpy() for i in range(len(test_ds))])
testY = np.array([test_ds[i][1] for i in range(len(test_ds))])

In [ ]:
from torch.utils.data import DataLoader


In [ ]:
# weight_decay = 1e-4
# max_lr=0.001

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(2048,256)
        #self.lin2 = nn.Linear(512, 256)
        #self.lin3 = nn.Linear(256,128)
        self.lin4=nn.Linear(256,64)
        self.lin5=nn.Linear(64,1)
        self.drop = nn.Dropout(0.1)
        
    def forward(self, inp):
        out = self.drop(F.relu(self.lin1(torch.squeeze(inp.float()))))
        #out = F.relu(self.lin2(out))
        #out = self.drop(F.relu(self.lin3(out)))
        out = self.drop(F.relu(self.lin4(out)))
        out = torch.sigmoid(self.lin5(out))
        return out

In [ ]:
model = Model().to(device)
opt = torch.optim.Adam(model.parameters(),lr=1e-4)
loss_fn = nn.MSELoss()

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)
    loss_list=[]
    model.train()
    
#     print('done')
    for index,(x,y) in enumerate(dataloader):
        
#         print(x,y)
#         print(len(y))
#         print(torch.Tensor(y).shape)
        x,y = x.to(device), y.to(device)
    

#         Compute prediction error
        pred = model(x)
        print(torch.squeeze(pred).shape, y.shape)
        loss = loss_fn(torch.squeeze(pred),y.float())
        loss_list+=[loss]
        loss.backward()

        # Backpropagation
        optimizer.step()
        optimizer.zero_grad()
        
        
    return((sum(loss_list))/len(loss_list))
        
    
    #print(dataloader)

In [ ]:
from torch.utils.data.dataloader import DataLoader

train_dl = DataLoader(train_ds, batch_size =128, shuffle = True)


test_dl = DataLoader(test_ds, batch_size = 32,shuffle = True)

In [ ]:
final_list_loss=np.array([])
for _ in range(320):
    final_list_loss=np.append(final_list_loss,np.array(train(train_dl, model, loss_fn, opt).cpu().detach().numpy()))
#print(final_list_loss)  

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
loss=final_list_loss
y=[i+1 for i in range(320)]
plt.plot(y,loss)
plt.ylabel("Loss")
plt.xlabel("Epochs")

In [ ]:
preds = []
for img, label in test_ds:
    img = img.to(device)
    with torch.no_grad():
        model.eval()
        pred = model(img.unsqueeze(0))
#         print(type(pred))
    preds.append(pred)
print(preds)

In [ ]:
# for x,y in train_dl:
#     print(x)
#     print(y)
#     break

SVR CLASSIFIER

In [ ]:
# import sklearn
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler

In [ ]:
# regr = SVR(kernel = 'rbf', C=1, epsilon=0.02, verbose=True)
# regr.fit(trainX, trainY)

In [ ]:
# from scipy import stats
# preds = regr.predict(testX)
# stats.spearmanr(preds, testY)

In [ ]:
from scipy import stats
x,y=stats.spearmanr(preds, testY)
print(x)

In [ ]:
# import scipy.io
# mat = scipy.io.loadmat('../input/lastiqa/AllMOS_release.mat')
# print(mat)

In [ ]:
torch.save(model.state_dict(), 'IQAweights.pth')

In [ ]:
correlation=[]
for _ in range(100):
    
    train_ds,test_ds=train_test_split(dataset,test_size=0.2)
    #print(train_ds)

    trainX = np.array([train_ds[i][0].cpu().numpy() for i in range(len(train_ds))])
    trainY = np.array([train_ds[i][1] for i in range(len(train_ds))])

    testX = np.array([test_ds[i][0].cpu().numpy() for i in range(len(test_ds))])
    testY = np.array([test_ds[i][1] for i in range(len(test_ds))])
    preds = []
    for img, label in test_ds:
        img = img.to(device)
        with torch.no_grad():
            model.eval()
            pred = model(img.unsqueeze(0))[0].item()
        preds.append(pred)
    x,y=stats.spearmanr(preds, testY)
    correlation+=[x]
    #print(y)
print(correlation)

In [ ]:
correlation_avg=(sum(correlation))/len(correlation)
print(correlation_avg)#0.7041889322260991